1. Goto http://lims2/custom_reports
2. Goto Cell Types cell report card
3. Click Generate
4. Click Get Report
5. Select Save As
6. Move custom_report.csv to working folder

In [1]:
import allensdk_lims.internal.core.lims_utilities as lu
from allensdk.core.nwb_data_set import NwbDataSet
from allensdk.ephys.extract_cell_features import extract_cell_features

import os
import numpy as np
import pandas as pd
from collections import defaultdict


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')



ImportError: No module named allensdk_lims.internal.core.lims_utilities

In [2]:
#SQL query from David
query = """
select wkf.storage_directory || wkf.filename as nwb_path, sp.id as specimen_id, sp.name as specimen_name
from well_known_files wkf 
join ephys_roi_results err on wkf.attachable_id = err.id
join specimens sp on sp.ephys_roi_result_id = err.id
where wkf.well_known_file_type_id = 475137571
and err.workflow_state = 'manual_passed'

"""
nwb_files = lu.query(query)
cell_index = { c['specimen_id']: c for c in nwb_files}

nwb_df = pd.DataFrame(nwb_files)

In [6]:
alm_df.head()

,project,cell_specimen_id,cell_specimen_name,slice_histology_well,slice_biocytin_well,patched_cell_container,ephys_roi_result_id,recording_date,tcp_sop,ephys_qc,...,v_rest,rheobase_sweep,latency,instant_sweep,rheobase_i,noise_1,noise_2,hero_rate,fast_trough,hero_sweep
572,T504,519266240,Chrna2-Cre_OE25;Ai14(BT)-245773.07.06.01,Ephys_160516_04_C06_VER2,BIOCYTIN_160524_04_C06,NaN,519266092,2016-05-18 15:20:15,ALM Sagittal 107,manual_passed,...,None,None,None,None,None,None,None,None,None,None
576,T504,518749558,Chrna2-Cre_OE25;Ai14(BT)-245774.07.06.01,Ephys_160516_04_A04_VER2,BIOCYTIN_160524_04_A04,NaN,518749510,2016-05-17 10:59:41,ALM Sagittal 107,manual_passed,...,None,None,None,None,None,None,None,None,None,None
577,T504,518743384,Chrna2-Cre_OE25;Ai14(BT)-245774.08.06.01,Ephys_160516_04_A03_VER2,BIOCYTIN_160524_04_A03,NaN,518743344,2016-05-17 09:47:11,ALM Sagittal 107,manual_passed,...,None,None,None,None,None,None,None,None,None,None
578,T504,518736746,Chrna2-Cre_OE25;Ai14(BT)-245774.09.06.01,Ephys_160516_04_A01_VER2,BIOCYTIN_160524_04_A01,NaN,518736685,2016-05-17 09:26:00,ALM Sagittal 107,manual_passed,...,None,None,None,None,None,None,None,None,None,None
580,T504,519773532,Chrna2-Cre_OE25;Ai14(BT)-245775.06.06.01,Ephys_160516_05_B04_VER2,BIOCYTIN_160524_05_B04,NaN,519773366,2016-05-20 14:41:15,ALM Sagittal 107,manual_passed,...,None,None,None,None,None,None,None,None,None,None


In [13]:
query2 = """
select es.sweep_number, est.description as stimulus_code, estt.name as stimulus_type from ephys_sweeps es
join ephys_stimuli est on es.ephys_stimulus_id = est.id
join ephys_stimulus_types estt on estt.id = est.ephys_stimulus_type_id
where es.specimen_id = """

custom_report = pd.read_csv('custom_report.csv')
alm_df = custom_report[ custom_report['project'] == 'T504']
alm_df = alm_df[alm_df['ephys_qc'] == 'manual_passed']

new_columns = ['tau','sag','ri','sag_amp','ISI_initial/ISI_mean','adaptation_index',
               'capacitance','AP_amp','upstroke_downstroke_ratio','v_rest',
               'rheobase_sweep','latency','instant_sweep','rheobase_i', 
               'noise_1','noise_2','hero_rate','fast_trough','hero_sweep']

for i in new_columns:
    alm_df[i] = None

for i,cell in enumerate(alm_df['cell_specimen_id']):
    alm_df['nwb_filename'].iloc[i] = nwb_df['nwb_path'][nwb_df.specimen_id == cell].iloc[0]
    alm_df['nwb_filename'].iloc[i] = lu.linux_to_windows(alm_df['nwb_filename'].iloc[i])
    file_name =  alm_df['nwb_filename'].iloc[i]
    data_set = NwbDataSet(file_name)
    #print file_name
    sweeps = lu.query(query2+str(cell))
    sweep_numbers = defaultdict(list)
    for sweep in sweeps:
        sweep_numbers[sweep['stimulus_type']].append(sweep['sweep_number'])
        
    try:
        cell_features = extract_cell_features(NwbDataSet(file_name),
                                      sweep_numbers['Ramp'],
                                      sweep_numbers['Short Square'],
                                      sweep_numbers['Long Square'])
    
    
        alm_df['tau'].iloc[i] = cell_features['long_squares']['tau']
        alm_df['sag'].iloc[i] = cell_features['long_squares']['sag']
        alm_df['ri'].iloc[i] = cell_features['long_squares']['input_resistance']
        alm_df['sag_amp'].iloc[i] = cell_features['long_squares']['v_baseline'] - (cell_features['long_squares']['vm_for_sag']*cell_features['long_squares']['sag'])
        alm_df['ISI_initial/ISI_mean'].iloc[i] = cell_features['long_squares']['hero_sweep']['first_isi']/cell_features['long_squares']['hero_sweep']['mean_isi']
        alm_df['adaptation_index'].iloc[i] = cell_features['long_squares']['hero_sweep']['adapt']
        alm_df['capacitance'].iloc[i] = cell_features['long_squares']['tau']/cell_features['long_squares']['input_resistance']*1000
        alm_df['AP_amp'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['peak_v'] - cell_features['long_squares']['hero_sweep']['spikes'][0]['threshold_v']
        alm_df['upstroke_downstroke_ratio'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['spikes'][0]['upstroke_downstroke_ratio']
        alm_df['v_rest'].iloc[i] = cell_features['long_squares']['v_baseline']
        alm_df['rheobase_sweep'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['id']
        alm_df['latency'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['latency']
        alm_df['instant_sweep'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['id']
        alm_df['rheobase_i'].iloc[i] = cell_features['long_squares']['rheobase_i']
        try:
            alm_df['noise_1'].iloc[i] = sweep_numbers['Noise 1'][0]
            alm_df['noise_2'].iloc[i] = sweep_numbers['Noise 2'][0]
        except IndexError:
            pass
        alm_df['hero_rate'].iloc[i] = cell_features['long_squares']['hero_sweep']['avg_rate']
        spikes = cell_features['long_squares']['hero_sweep']['spikes']
        alm_df['fast_trough'].iloc[i] = np.mean([spikes[f]['fast_trough_v'] for f in range(len(spikes))])
        alm_df['hero_sweep'].iloc[i] = cell_features['long_squares']['hero_sweep']['id']
    except TypeError:
        pass
    print cell

519266240
518749558
518743384
518736746
519773532
519774371
519788845
518755967
520024429
520025791
519220689
519245130
519249314
521008506
521011491
529693591
530356475
519586281
519589169
519589782
523642708
523782484
523463678
523451532
523463900
531345499
531344295
531346266
523241382
523236690
523248935
523245775
523242198
523243228
523241481
523496035
523508694
520737170
520739673
525857876
525871774
526194846
526555863
526555815
526569450
527079140
526929030
526922424
519576389
519578829
526510091
526491738
541229840
541225512
541229574
540748047
540742492
540751927
541029553
541026126
544506266
544490027
544507584
544474304
543873287
543869784
545471941
545470913
545472779
545464284
545327293
545327333
545345208
545083578
545093468
544872480
544890942
545583697
545560746
545568826
545570919
546661234
546678466
546690892
546392620
546336203
546319969
546319225
546328190
546332863
546333535
546336651
548351058
552569700
552583458
552584949
521450010
521487760
522437149
522438949


In [14]:
alm_df.tail()

,project,cell_specimen_id,cell_specimen_name,slice_histology_well,slice_biocytin_well,patched_cell_container,ephys_roi_result_id,recording_date,tcp_sop,ephys_qc,...,v_rest,rheobase_sweep,latency,instant_sweep,rheobase_i,noise_1,noise_2,hero_rate,fast_trough,hero_sweep
6227,T504,552329723,Vip-IRES-Cre;Ai14(IVSCC)-276117.10.06.01,EPhys_161010_01_C05_M,BIOCYTIN_161018_01_C05,NaN,552329707,2016-10-13 15:39:43,ALM Sagittal 107,manual_passed,...,-70.70315,35,0.27334,10,40,42,43,46,-55.34375,29
6228,T504,552416754,Vip-IRES-Cre;Ai14(IVSCC)-276122.07.06.01,EPhys_161010_01_C06_M,BIOCYTIN_161018_01_C06,NaN,552416729,2016-10-14 09:20:07,ALM Sagittal 107,manual_passed,...,-59.01106,38,0.2732,16,20,46,47,40,-44.23281,44
6230,T504,552417215,Vip-IRES-Cre;Ai14(IVSCC)-276122.10.06.01,EPhys_161010_01_D01_M,BIOCYTIN_161018_01_D01,NaN,552417191,2016-10-14 09:43:16,ALM Sagittal 107,manual_passed,...,-63.43586,27,0.27334,14,50,44,45,17,-44.01103,29
6231,T504,552428561,Vip-IRES-Cre;Ai14(IVSCC)-276122.11.06.01,EPhys_161010_01_D02_M,BIOCYTIN_161018_01_D02,NaN,552428500,2016-10-14 10:39:26,ALM Sagittal 107,manual_passed,...,-67.91476,25,0.2732,13,30,40,42,15,-49.67709,27
6232,T504,552457447,Vip-IRES-Cre;Ai14(IVSCC)-277405.07.06.01,EPhys_161010_04_A01_M,BIOCYTIN_161018_04_A01,NaN,552457394,2016-10-14 15:49:19,ALM Sagittal 107,manual_passed,...,-69.96629,33,0.27312,15,110,50,51,7,-44.80357,35


In [15]:
for row in range(len(alm_df)):
    for title in alm_df.columns:
        if type(alm_df[title].iloc[row]) == tuple:
            alm_df[title].iloc[row] = alm_df[title].iloc[row][0]

In [42]:
alm_df.head()

,project,cell_specimen_id,cell_specimen_name,slice_histology_well,slice_biocytin_well,patched_cell_container,ephys_roi_result_id,recording_date,tcp_sop,ephys_qc,...,has_glif1,has_glif2,has_glif3,has_glif4,has_glif5,has_biophysical,has_active_dendrite,bfi_filename,nwb_filename,swc_filename
572,T504,519266240,Chrna2-Cre_OE25;Ai14(BT)-245773.07.06.01,Ephys_160516_04_C06_VER2,BIOCYTIN_160524_04_C06,NaN,519266092,2016-05-18 15:20:15,ALM Sagittal 107,manual_passed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/projects/whbi/vol1/prod5/specimen_519216896/1...,\\titan\cns\mousecelltypes\prod757\Ephys_Roi_R...,/projects/whbi/vol1/prod15/specimen_519266240/...
576,T504,518749558,Chrna2-Cre_OE25;Ai14(BT)-245774.07.06.01,Ephys_160516_04_A04_VER2,BIOCYTIN_160524_04_A04,NaN,518749510,2016-05-17 10:59:41,ALM Sagittal 107,manual_passed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/projects/whbi/vol1/prod4/specimen_518721739/1...,\\titan\cns\mousecelltypes\prod753\Ephys_Roi_R...,NaN
577,T504,518743384,Chrna2-Cre_OE25;Ai14(BT)-245774.08.06.01,Ephys_160516_04_A03_VER2,BIOCYTIN_160524_04_A03,NaN,518743344,2016-05-17 09:47:11,ALM Sagittal 107,manual_passed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/projects/whbi/vol1/prod4/specimen_518721831/1...,\\titan\cns\mousecelltypes\prod753\Ephys_Roi_R...,NaN
578,T504,518736746,Chrna2-Cre_OE25;Ai14(BT)-245774.09.06.01,Ephys_160516_04_A01_VER2,BIOCYTIN_160524_04_A01,NaN,518736685,2016-05-17 09:26:00,ALM Sagittal 107,manual_passed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/projects/whbi/vol1/prod4/specimen_518721816/1...,\\titan\cns\mousecelltypes\prod753\Ephys_Roi_R...,NaN
580,T504,519773532,Chrna2-Cre_OE25;Ai14(BT)-245775.06.06.01,Ephys_160516_05_B04_VER2,BIOCYTIN_160524_05_B04,NaN,519773366,2016-05-20 14:41:15,ALM Sagittal 107,manual_passed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/projects/whbi/vol1/prod5/specimen_519753640/1...,\\titan\cns\mousecelltypes\prod761\Ephys_Roi_R...,NaN


In [17]:
alm_df.to_csv('alm_df.csv')

In [1]:
import allensdk_lims.internal.core.lims_utilities as lu
from allensdk.core.nwb_data_set import NwbDataSet
from allensdk.ephys.extract_cell_features import extract_cell_features

import os
import numpy as np
import pandas as pd
from collections import defaultdict



%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

#SQL query from David
query = """
select wkf.storage_directory || wkf.filename as nwb_path, sp.id as specimen_id, sp.name as specimen_name
from well_known_files wkf 
join ephys_roi_results err on wkf.attachable_id = err.id
join specimens sp on sp.ephys_roi_result_id = err.id
where wkf.well_known_file_type_id = 475137571
and err.workflow_state = 'manual_passed'

"""
nwb_files = lu.query(query)
cell_index = { c['specimen_id']: c for c in nwb_files}

nwb_df = pd.DataFrame(nwb_files)

query2 = """
select es.sweep_number, est.description as stimulus_code, estt.name as stimulus_type from ephys_sweeps es
join ephys_stimuli est on es.ephys_stimulus_id = est.id
join ephys_stimulus_types estt on estt.id = est.ephys_stimulus_type_id
where es.specimen_id = """

custom_report = pd.read_csv('custom_report.csv')
v1_df = custom_report
v1_df = v1_df[v1_df['ephys_qc'] == 'manual_passed']

new_columns = ['tau','sag','ri','sag_amp','ISI_initial/ISI_mean','adaptation_index',
               'capacitance','AP_amp','upstroke_downstroke_ratio','v_rest',
               'rheobase_sweep','latency','instant_sweep','rheobase_i', 
               'noise_1','noise_2','hero_rate','fast_trough','hero_sweep']

for i in new_columns:
    v1_df[i] = None

for i,cell in enumerate(v1_df['cell_specimen_id']):
    v1_df['nwb_filename'].iloc[i] = nwb_df['nwb_path'][nwb_df.specimen_id == cell].iloc[0]
    v1_df['nwb_filename'].iloc[i] = lu.linux_to_windows(v1_df['nwb_filename'].iloc[i])
    file_name =  v1_df['nwb_filename'].iloc[i]
    data_set = NwbDataSet(file_name)
    #print file_name
    sweeps = lu.query(query2+str(cell))
    sweep_numbers = defaultdict(list)
    for sweep in sweeps:
        sweep_numbers[sweep['stimulus_type']].append(sweep['sweep_number'])
        
    try:
        cell_features = extract_cell_features(NwbDataSet(file_name),
                                      sweep_numbers['Ramp'],
                                      sweep_numbers['Short Square'],
                                      sweep_numbers['Long Square'])
    
    
        v1_df['tau'].iloc[i] = cell_features['long_squares']['tau']
        v1_df['sag'].iloc[i] = cell_features['long_squares']['sag']
        v1_df['ri'].iloc[i] = cell_features['long_squares']['input_resistance']
        v1_df['sag_amp'].iloc[i] = cell_features['long_squares']['v_baseline'] - (cell_features['long_squares']['vm_for_sag']*cell_features['long_squares']['sag'])
        #alm_df['ISI_initial/ISI_mean'].iloc[i] = cell_features['long_squares']['hero_sweep']['first_isi']/cell_features['long_squares']['hero_sweep']['mean_isi']
        v1_df['adaptation_index'].iloc[i] = cell_features['long_squares']['hero_sweep']['adapt']
        v1_df['capacitance'].iloc[i] = cell_features['long_squares']['tau']/cell_features['long_squares']['input_resistance']*1000
        v1_df['AP_amp'].iloc[i] = cell_features['long_squares']['hero_sweep']['spikes'][0]['peak_v'] - cell_features['long_squares']['hero_sweep']['spikes'][0]['threshold_v']
        v1_df['upstroke_downstroke_ratio'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['spikes'][0]['upstroke_downstroke_ratio']
        v1_df['v_rest'].iloc[i] = cell_features['long_squares']['v_baseline']
        v1_df['rheobase_sweep'].iloc[i] = cell_features['long_squares']['rheobase_sweep']['id']
        v1_df['latency'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['latency']
        v1_df['instant_sweep'].iloc[i] = cell_features['short_squares']['common_amp_sweeps'][0]['id']
        v1_df['rheobase_i'].iloc[i] = cell_features['long_squares']['rheobase_i']
        v1_df['hero_rate'].iloc[i] = cell_features['long_squares']['hero_sweep']['avg_rate']
        spikes = cell_features['long_squares']['hero_sweep']['spikes']
        v1_df['fast_trough'].iloc[i] = np.mean([spikes[f]['fast_trough_v'] for f in range(len(spikes))])
        v1_df['hero_sweep'].iloc[i] = cell_features['long_squares']['hero_sweep']['id']
    except TypeError:
        pass
    print cell

for row in range(len(v1_df)):
    for title in v1_df.columns:
        if type(v1_df[title].iloc[row]) == tuple:
            v1_df[title].iloc[row] = v1_df[title].iloc[row][0]

v1_df.to_csv('all_features.csv')

505795846
505798354
505804754


KeyboardInterrupt: 

In [2]:
v1_df.head()

,project,cell_specimen_id,cell_specimen_name,slice_histology_well,slice_biocytin_well,patched_cell_container,ephys_roi_result_id,recording_date,tcp_sop,ephys_qc,...,v_rest,rheobase_sweep,latency,instant_sweep,rheobase_i,noise_1,noise_2,hero_rate,fast_trough,hero_sweep
448,T301,505795846,234356.04.02.01,Ephys_160307_01_A04,BIOCYTIN_160315_01_A04,NaN,505795842,2016-03-07 10:36:10,V1 Coronal 107,manual_passed,...,-76,41,0.27336,28,130,None,None,3,-46.48958,43
450,T301,505798354,234356.05.02.01,Ephys_160307_01_A05,BIOCYTIN_160315_01_A05,NaN,505798350,2016-03-07 11:03:35,V1 Coronal 107,manual_passed,...,-72.2231,36,0.27222,18,50,None,None,17,-47.20221,29
452,T301,505804754,234356.06.02.01,Ephys_160307_01_B01,BIOCYTIN_160315_01_B01,NaN,505804750,2016-03-07 12:12:59,V1 Coronal 107,manual_passed,...,-74.15828,29,0.27284,20,50,None,None,11,-50.27273,31
469,T504x,504127978,Cart-IRES2-Cre;Ai14(IVSCC)-231003.04.03.01,Ephys_ALM_160225_A05,BIOCYTIN_160308_04_A05,NaN,504127972,2016-02-25 14:36:59,ALM Coronal 56,manual_passed,...,None,None,None,None,None,None,None,None,None,None
473,T301x,504110300,Cart-IRES2-Cre;Ai14(IVSCC)-231852.05.02.01,Ephys_ALM_160225_A03,BIOCYTIN_160308_04_A03,NaN,504110298,2016-02-25 11:36:19,V1 Coronal 90,manual_passed,...,None,None,None,None,None,None,None,None,None,None
